In [2]:
import pandas as pd
import os
import numpy as np

fp = '../data/processed/S2324_all_play-by-play.csv'
df = pd.read_csv(fp)

df.sample(5)

,game_id,period,clock,home,scoreHome,away,scoreAway,playerNameI,teamTricode,description,actionType,subType,xLegacy,yLegacy,shotDistance,isFieldGoal,shotVal,scoreVal,location
148744,S2324-G0973,1,05:54,LAL,NaN,GSW,NaN,R. Hachimura,LAL,Hachimura REBOUND (Off:0 Def:1),Rebound,Unknown,0,0,0,0,NaN,NaN,h
38100,S2324-G0587,3,10:14,MIA,NaN,ATL,NaN,J. Johnson,ATL,Johnson Lost Ball Turnover (P2.T7),Turnover,Lost Ball,0,0,0,0,NaN,NaN,v
176266,S2324-G1129,1,06:23,MEM,NaN,PHI,NaN,G. Jackson II,MEM,Jackson Lost Ball Turnover (P1.T2),Turnover,Lost Ball,0,0,0,0,NaN,NaN,h
51342,S2324-G0987,1,04:24,UTA,NaN,MIN,NaN,C. Sexton,UTA,MISS Sexton 4' Driving Dunk,Missed Shot,Driving Dunk Shot,37,11,4,1,2.0,0.0,h
83557,S2324-G0379,1,03:58,OKC,NaN,LAC,NaN,P. George,LAC,George S.FOUL (P2.T4) (M.Smith),Foul,Shooting,0,0,0,0,NaN,NaN,v


In [34]:
import numpy as np
teams, players, actions, times = [], [], [], []

cols = [teams, players, actions, times]
colNames = ['home', 'playerNameI', 'actionType', 'clock']


for i, col in enumerate(cols):
  cur = df[colNames[i]].unique()
  cols[i] = pd.Series(cur).sort_values().dropna().reset_index(drop=True)


teams = cols[0]
players = cols[1]
actions = cols[2]
times = cols[3]

In [35]:
for action in actions:
  print(action)

Ejection
Foul
Free Throw
Instant Replay
Jump Ball
Made Shot
Missed Shot
Rebound
Substitution
Timeout
Turnover
Violation
period


## **Issue: Duplicates in** `actionType`
Goal: Figure out the difference between these duplicate entries. Good chance that they come in pairs and just a new row corresponding to the same event.

In [21]:
foul1 = actions[1]    # Gets 'Foul' with normal whitespace
foul2 = actions[2]    # Gets 'Foul' with extra whitespace

print(f"foul1 length: {len(foul1)} \t foul2 length: {len(foul2)}")

foul1 length: 4 	 foul2 length: 40


### **Getting a sample of 5 rows for each foul type**
Checking for differences between the two.

In [27]:
foul1_df = df[df['actionType'] == foul1]    # 4 character 'Foul'
foul1_df.sample(5)

,game_id,period,clock,home,scoreHome,away,scoreAway,playerNameI,teamTricode,description,actionType,subType,xLegacy,yLegacy,shotDistance,isFieldGoal,shotVal,scoreVal,location
481166,S2324-G0842,4,09:38,POR,NaN,MIA,NaN,J. Butler,MIA,Butler S.FOUL (P3.T1) (B.Barnaky),Foul,Shooting,0,0,0,0,NaN,NaN,v
172168,S2324-G0446,3,07:55,NOP,NaN,LAL,NaN,Z. Williamson,NOP,Williamson P.FOUL (P3.T3) (M.Dagher),Foul,Personal,0,0,0,0,NaN,NaN,h
465051,S2324-G0209,4,00:33.800000,IND,NaN,ORL,NaN,A. Schofield,ORL,"Double Technical - Schofield, Jackson (B.Taylor)",Foul,Double Technical,0,0,0,0,NaN,NaN,NaN
256332,S2324-G0001,3,00:23.300000,IND,NaN,CLE,NaN,I. Okoro,CLE,Okoro OFF.Foul (P1) (B.Barnaky),Foul,Offensive,0,0,0,0,NaN,NaN,v
113197,S2324-G0586,2,10:37,BOS,NaN,DEN,NaN,P. Watson,DEN,Watson S.FOUL (P1.T1) (K.Cutler),Foul,Shooting,0,0,0,0,NaN,NaN,v


In [28]:
foul2_df = df[df['actionType'] == foul2]    # 40 character 'Foul'
foul2_df.sample(5)

,game_id,period,clock,home,scoreHome,away,scoreAway,playerNameI,teamTricode,description,actionType,subType,xLegacy,yLegacy,shotDistance,isFieldGoal,shotVal,scoreVal,location
534104,S2324-G0253,2,00:06.600000,DET,NaN,WAS,NaN,D. Avdija,WAS,Avdija Non-Unsportsmanlike Tech Foul - Floppin...,Foul,Flopping,0,0,0,0,NaN,NaN,v
524909,S2324-G0357,4,10:02,UTA,NaN,BKN,NaN,J. Collins,UTA,Collins Transition Take Foul (P2.T1) (N.Sago),Foul,Transition Take,0,0,0,0,NaN,NaN,h
247639,S2324-G0308,3,06:39,PHX,NaN,GSW,NaN,C. Metu,PHX,Metu Transition Take Foul (P2.T3) (B.Forte),Foul,Transition Take,0,0,0,0,NaN,NaN,h
55846,S2324-G1178,2,01:58,MIN,NaN,ATL,NaN,A. Edwards,MIN,Edwards Transition Take Foul (P4.PN) (A.Smith),Foul,Transition Take,0,0,0,0,NaN,NaN,h
72109,S2324-G0905,3,09:45,PHX,NaN,TOR,NaN,K. Olynyk,TOR,Olynyk Transition Take Foul (P3.T2) (E.Scott),Foul,Transition Take,0,0,0,0,NaN,NaN,v


In [29]:
print(f"4-char fouls: {len(foul1_df)}, \t 40-char fouls: {len(foul2_df)}")

4-char fouls: 47416, 	 40-char fouls: 230


In [32]:
foul1_df['subType'].unique()

array(['Offensive', 'Shooting', 'Loose Ball', 'Offensive Charge',
       'Personal', 'Away From Play', 'Defense 3 Second', 'Technical',
       'Personal Take', 'Flagrant Type 2', 'Double Technical',
       'Flagrant Type 1', 'Hanging Technical', 'Clear Path',
       'Delay Technical', 'Double Personal', 'Excess Timeout Technical',
       'Too Many Players Technical', 'Non-Unsportsmanlike Technical'],
      dtype=object)

In [31]:
foul2_df['subType'].unique()

array(['Flopping', 'Transition Take', 'Bench'], dtype=object)

In [33]:
# Iterate through 'df' and strip whitespace from the 'actionType' column entries

df['actionType'] = df['actionType'].str.strip()

array(['period', 'Jump Ball', 'Missed Shot', 'Rebound', 'Turnover', nan,
       'Made Shot', 'Foul', 'Free Throw', 'Timeout', 'Substitution',
       'Instant Replay', 'Violation', 'Ejection'], dtype=object)

In [36]:
df['actionType'].unique()

array(['period', 'Jump Ball', 'Missed Shot', 'Rebound', 'Turnover', nan,
       'Made Shot', 'Foul', 'Free Throw', 'Timeout', 'Substitution',
       'Instant Replay', 'Violation', 'Ejection'], dtype=object)

array(['NYK', 'ORL', 'UTA', 'MEM', 'BOS', 'LAL', 'MIA', 'SAC', 'CHI',
       'ATL', 'HOU', 'TOR', 'CLE', 'GSW', 'DET', 'LAC', 'POR', 'NOP',
       'BKN', 'PHI', 'MIN', 'WAS', 'CHA', 'IND', 'SAS', 'MIL', 'OKC',
       'DEN', 'DAL', nan, 'PHX'], dtype=object)

In [39]:
def save_by_team(fp):
  """
  Function saves the dataframe to a csv file for each player in the dataframe. The csv files are saved in the 'processed' folder.

  Args:
    fp (str): File path to save the csv files to

  Returns:
    None
  """
  try:
    df = pd.read_csv(fp)
    teams = pd.Series(df['home'].unique()).sort_values().dropna().reset_index(drop=True)
    print(teams)

    for team in teams:
      team_df = df[(df['home'] == team) | (df['away'] == team)]
      team_df.to_csv(f"../data/processed/team/{team}_games.csv", index=False)

    print(f"Successfully saved {len(teams)} files containing team-specific data.")

  except:
    print(f"Error: Could not save csv files for each team in the dataframe.")

fp = '../data/processed/S2324_all_play-by-play.csv'
save_by_team(fp)

0     ATL
1     BKN
2     BOS
3     CHA
4     CHI
5     CLE
6     DAL
7     DEN
8     DET
9     GSW
10    HOU
11    IND
12    LAC
13    LAL
14    MEM
15    MIA
16    MIL
17    MIN
18    NOP
19    NYK
20    OKC
21    ORL
22    PHI
23    PHX
24    POR
25    SAC
26    SAS
27    TOR
28    UTA
29    WAS
dtype: object
Successfully saved 30 files containing team-specific data.


In [68]:
def save_by_player(fp):
  """
  Function saves the dataframe to a csv file for each player in the dataframe. The csv files are saved in the 'processed' folder.

  Args:
    fp (str): File path to save the csv files to

  Returns:
    None
  """
  try:
    df = pd.read_csv(fp)
    players = pd.Series(df['playerNameI'].unique()).sort_values().dropna().reset_index(drop=True)


    for player in players:
      player_name = str(player).replace(' ', '')
      player_df = df[df['playerNameI'] == player]

      # teams = pd.Series(df[df['teamTricode']].unique()).sort_values().dropna().reset_index(drop=True)

      # for team in teams:
      #   player_team_df = player_df[player_df['teamTricode'] == team]
      #   player_df.to_csv(f"../data/processed/player/{player}/{team}_{player}_games.csv", index=False)
      player_df.to_csv(f"../data/processed/player/{player_name}_play-by-play.csv", index=False)


    print(f"Successfully saved {len(players)} files containing player-specific data.")

  except:
    print(f"Error: Could not save csv files for each team in the dataframe.")

fp = '../data/processed/S2324_all_play-by-play.csv'
save_by_player(fp)

../data/processed/S2324_all_play-by-play.csv
(598705, 19)
825
A.Avent
Player df: (5, 19)
A.Bailey
Player df: (50, 19)
A.Black
Player df: (809, 19)
A.Bonner
Player df: (1, 19)
A.Bowie
Player df: (5, 19)
A.Brooks
Player df: (81, 19)
A.Burks
Player df: (1166, 19)
A.Caldwell
Player df: (4, 19)
A.Carr
Player df: (2, 19)
A.Caruso
Player df: (1661, 19)
A.Coffey
Player df: (860, 19)
A.Davis
Player df: (3538, 19)
A.DeClercq
Player df: (2, 19)
A.Dosunmu
Player df: (1633, 19)
A.Drummond
Player df: (1977, 19)
A.Edwards
Player df: (3134, 19)
A.Flagler
Player df: (7, 19)
A.Fudge
Player df: (32, 19)
A.Funk
Player df: (7, 19)
A.Gill
Player df: (422, 19)
A.Gilliam
Player df: (4, 19)
A.Gordon
Player df: (1999, 19)
A.Green
Player df: (418, 19)
A.Griffin
Player df: (119, 19)
A.Hagans
Player df: (257, 19)
A.Hardaway
Player df: (3, 19)
A.Harvey
Player df: (3, 19)
A.Henderson
Player df: (2, 19)
A.Holiday
Player df: (968, 19)
A.Horford
Player df: (1297, 19)
A.Houston
Player df: (1, 19)
A.JacksonJr.
Player df:

In [43]:
df.size

11375395

In [50]:
df = pd.read_csv('../data/processed/S2324_all_play-by-play.csv')
df.shape

(598705, 19)

In [88]:
def save_by_teamplayer(fp):
  """
  Function saves the dataframe to a csv file for each player in the dataframe. The csv files are saved in the 'processed' folder.

  Args:
    fp (str): File path to save the csv files to

  Returns:
    None
  """
  try:
    df = pd.read_csv(fp)
    print(df.shape)
    players = pd.Series(df['playerNameI'].unique()).sort_values().dropna().reset_index(drop=True)


    for player in players:
      player_name = str(player).replace(' ', '')
      player_df = df[df['playerNameI'] == player]
      print(player_df.shape)

      teams = pd.Series(player_df['teamTricode'].unique())
      # teams = pd.Series(df['teamTricode']).unique().sort_values().dropna().reset_index(drop=True)
      print(len(teams))

      for team in teams:
        if isinstance(team, str):
          print(f"working on saving: {team}")
          player_team_df = player_df[player_df['teamTricode'] == team]
          print(player_team_df.shape)
          player_team_df.to_csv(f"../data/processed/teamplayer/{team}_{player}_games.csv", index=False)



    print(f"Successfully saved {len(players)} files containing player-specific data.")

  except:
    print(f"Error: Could not save csv files for each team in the dataframe.")

fp = '../data/processed/S2324_all_play-by-play.csv'
save_by_teamplayer(fp)

(598705, 19)
(5, 19)
1
(50, 19)
1
working on saving: CHA
(50, 19)
(809, 19)
1
working on saving: ORL
(809, 19)
(1, 19)
1
(5, 19)
1
(81, 19)
1
working on saving: BKN
(81, 19)
(1166, 19)
2
working on saving: NYK
(310, 19)
working on saving: DET
(856, 19)
(4, 19)
1
(2, 19)
1
(1661, 19)
1
working on saving: CHI
(1661, 19)
(860, 19)
1
working on saving: LAC
(860, 19)
(3538, 19)
1
working on saving: LAL
(3538, 19)
(2, 19)
1
(1633, 19)
1
working on saving: CHI
(1633, 19)
(1977, 19)
1
working on saving: CHI
(1977, 19)
(3134, 19)
1
working on saving: MIN
(3134, 19)
(7, 19)
1
working on saving: OKC
(7, 19)
(32, 19)
2
working on saving: LAL
(12, 19)
working on saving: DAL
(20, 19)
(7, 19)
1
working on saving: CHI
(7, 19)
(422, 19)
1
working on saving: WAS
(422, 19)
(4, 19)
1
(1999, 19)
1
working on saving: DEN
(1999, 19)
(418, 19)
1
working on saving: MIL
(418, 19)
(119, 19)
1
working on saving: ATL
(119, 19)
(257, 19)
1
working on saving: POR
(257, 19)
(3, 19)
1
(3, 19)
1
(2, 19)
1
(968, 19)
1
w